In [1]:
from pprint import pprint

import numpy as np
import pandas as pd

import matplotlib.pyplot as plt
import seaborn as sns
import plotly.express as px

from utils import *
import fantraxAPI

# from fantraxAPI import *

import tools_cvf

In [2]:
leagueInfo = fantraxAPI.fetch_leagueInfo()

requesting league info. status code: 200


In [11]:
# create a dataframe of player info
df_playerList = pd.DataFrame(
    [[k, *v.values()] for k, v in leagueInfo["playerInfo"].items()],
    columns=["fantraxid", "pos_elig", "status"],
)
# append team details where we can (so far)
df_playerList["team"] = None
df_playerList.loc[df_playerList.status == "FA", "team"] = False
df_playerList.loc[df_playerList.status == "WW", "team"] = False
teamRosters = fantraxAPI.fetch_teamRosters()
for team in teamRosters["rosters"].keys():
    for fantraxid in [ri["id"] for ri in teamRosters["rosters"][team]["rosterItems"]]:
        df_playerList.loc[df_playerList.fantraxid == fantraxid, "team"] = team

# get the team info for each team
df_teamInfo = pd.DataFrame(
    [list(v.values()) for v in leagueInfo["teamInfo"].values()],
    columns=["division", "name", "id"],
)
# add the team info to players now that we have it
df_playerList = df_playerList.merge(
    df_teamInfo.add_prefix("team_"), how="left", left_on="team", right_on="team_id"
).drop(columns="team")

df_playerList[df_playerList.status == "T"]

requesting team rosters. status code: 200


,fantraxid,pos_elig,status,team_division,team_name,team_id
178,04b02,"OF,UT",T,AOA West,Salt Lake Fireflies,i1xlfdkplrs9xvi3
199,04azv,"3B,UT",T,AOA West,DeepFly Factory,h6kltku3lrsgambq
219,04auj,SP,T,AOA East,Amontillado Trowels,1770mj2elrsgambq
305,029si,SP,T,AOA West,DeepFly Factory,h6kltku3lrsgambq
378,04aea,SP,T,AOA East,Eagle Fang Baseball,0037qhi7lrsgambq
...,...,...,...,...,...,...
14751,000dp,SP,T,AOA West,DeepFly Factory,h6kltku3lrsgambq
15124,05aoc,SP,T,AOA West,DeepFly Factory,h6kltku3lrsgambq
15135,05ajg,"2B,UT,3B,SS",T,AOA West,Airsick Lowlanders,nrrvbec0lrsgambq
15235,04a2t,SP,T,AOA East,District Dingers,5om1y9lglrsgambq
